In [15]:
import jieba
import pandas as pd
import numpy as np
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
path = "D:"

In [16]:
# 获取停用词
stop_words = pd.read_csv(path + "\\GraduationDesign\\语料库\\stopwords.dat", delimiter="\t", header=None, quoting=3, encoding='utf-8')
stop_words = stop_words[0].tolist()

In [17]:
df = pd.read_csv(path + "\\GraduationDesign\\语料库\\客服语料\\整理后\\1.csv")
q_list = []
for doc in df.iloc[:,0]:
    q_list.append([word for word in jieba.cut(doc) if word not in stop_words and word !=' '])
with open(path + "\\GraduationDesign\\语料库\\vecDemo.txt", "w", encoding="utf-8") as f:
    for q in q_list:
        f.write(" ".join(q) + "\n")

In [18]:
model = word2vec.Word2Vec(LineSentence(path + "\\GraduationDesign\\语料库\\vecDemo.txt"), sg=0, vector_size=200, window=3, min_count=1, workers=8)

model.save(path + '\\GraduationDesign\\answerM\\models\\word2vec.model')

In [19]:
model = word2vec.Word2Vec.load(path + '\\GraduationDesign\\answerM\\models\\word2vec.model')
dic = model.wv.index_to_key
print(dic)
print(len(dic))


['订单', '商品', '想', '产品', '账户', '取消', '查询', '收到', '购买', '退货', '更改', '密码', '请问', '信息', '支付', '我该', '质量', '操作', '退款', '忘记', '网站', '付款', '做', '地址', '客服', '锁定', '状态', '配送', '发货', '修改', '优惠券', '想要', '损坏', '解决', '方式', '不', '送达', '时', '下单', '找到', '不符', '提供', '登录', '退换货', '申请', '查看', '未', '包裹', '情况', '描述', '公司', '政策', '支持', '时间', '物流', '成功', '我要', '缺货', '咨询', '显示', '详细信息', '余额', '发现', '失败', '解锁', '注册', '优惠', '中', '收货', '码', '多久', '账号', '丢失', '售后服务', '提交', '库存', '有没有', '查找', '过程', '优惠活动', '人员', '一件', '个人资料', '账', '到达', '错误', '确认', '积分', '瑕疵', '被盗', '运输', '获取', '告诉', '满意', '服务', '保修期', '投诉', '拒绝', '货', '符合', '到货', '活动', '缺陷', '保修', '历史记录', '会员', '促销', '技术支持', '邮件', '刚刚', '发票', '送货', '怎么回事', '商品质量', '找回', '重置', '新', '适合', '商品价格', '差价', '原因', '快递', '费用', '功能', '价格', '有误', '礼品', '退换', '运费', '详细', '国际', '参数', '选择', '请求', '寻求', '产品质量', '账号密码', '购物', '送到', '小心', '免费', '团队', '删除', '客户服务', '流程', '特定', '发生', '更新', '卡', '维修', '信用卡', '账单', '故障', '找', '多长时间', '变动', '怎样才能', '某款', '您好', '货到付款', '相关', '不到', '订购'

In [20]:
# 计算备选问题向量
doc_dict = {}
for i, doc in enumerate(q_list):
    doc_vec = []
    for word in doc:
        if word in dic:
            doc_vec.append(model.wv[word])
    doc_vec = np.mean(doc_vec, axis=0)
    doc_dict[i] = doc_vec

In [21]:
# 计算问题向量
question = "怎么注册新帐号啊？"
question = [word for word in jieba.cut(question) if word not in stop_words and word !=' ']
q_vec = []
for word in question:
    if word in dic:
        q_vec.append(model.wv[word])
q_vec = np.mean(q_vec, axis=0)

In [22]:
prob_ans = model.wv.cosine_similarities(q_vec, [doc_dict[i] for i in range(len(q_list))])
# 选出前三个最相似的问题
prob_ans = np.argsort(prob_ans)[-3:]
all_answer = df.iloc[:,1]
print("匹配到问题：", df.iloc[prob_ans[0],0])
print("回答：" + all_answer[prob_ans[0]])

匹配到问题： 我是新用户，如何注册？
回答：非常感谢您选择我们的服务！您可以通过我们的网站或应用程序进行注册。请填写必要的信息并创建账户即可完成注册。
